In [ ]:
############ example for a Alphalens tear sheet similar to
############ https://www.quantopian.com/tutorials/getting-started#lesson4
############ just works with zipline instead with quantopian.pipeline

from zipline.api import symbol

import zipline.algorithm as algo

import alphalens as al

from zipline.data import bundles

from zipline.data.data_portal import DataPortal

from zipline.pipeline import Pipeline
from zipline.pipeline.factors import AverageDollarVolume, Returns, RSI, SimpleMovingAverage
from zipline.pipeline.loaders import USEquityPricingLoader
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.engine import SimplePipelineEngine
from zipline.pipeline.filters import StaticAssets

from zipline.utils.calendars import get_calendar

from zipline.utils.run_algo import load_extensions

import numpy as np
import pandas as pd


#### need this to be able to access a custom bundle, would be nice if this would be included into zipline library one day
# Load extensions.py; this allows you access to custom bundles
import os
load_extensions(
    default=True,
    extensions=[],
    strict=True,
    environ=os.environ,
)

####### get_pricing is missing

def get_pricing(trading_calendar, assets, start_date, end_date, field='close'):
    # Set the given start and end dates to Timestamps. The frequency string C is used to
    # indicate that a CustomBusinessDay DateOffset is used
    end_dt = pd.Timestamp(end_date, tz='UTC', freq='C')
    start_dt = pd.Timestamp(start_date, tz='UTC', freq='C')

    # Get the locations of the start and end dates
    end_loc = trading_calendar.closes.index.get_loc(end_dt)
    start_loc = trading_calendar.closes.index.get_loc(start_dt)
    
    data_portal = DataPortal(bundle_data.asset_finder,
                         trading_calendar = trading_calendar,
                         first_trading_day = bundle_data.equity_daily_bar_reader.first_trading_day,
                         equity_daily_reader = bundle_data.equity_daily_bar_reader,
                         adjustment_reader = bundle_data.adjustment_reader)

    # return the historical data for the given window
    return data_portal.get_history_window(assets=assets, end_dt=end_dt, bar_count=end_loc - start_loc,
                                          frequency='1d',
                                          field=field,
                                          data_frequency='daily')
########################## done


bundle_data = bundles.load('sec_master_q')
# Set the trading calendar
trading_calendar = get_calendar('NYSE')




#################################

# Create a Pipeline engine ....... why can't i use just use
# this: factors = PipelineEngine.run_pipeline(initialize(universe),start_date=start_date,end_date=end_date) ?????

# Set the dataloader
pricing_loader = USEquityPricingLoader(bundle_data.equity_daily_bar_reader, bundle_data.adjustment_reader)

# Define the function for the get_loader parameter
def choose_loader(column):
    if column not in USEquityPricing.columns:
        raise Exception('Column not in USEquityPricing')
    return pricing_loader

# Create a Pipeline engine
engine = SimplePipelineEngine(get_loader = choose_loader,
                              calendar = trading_calendar.all_sessions,
                              asset_finder = bundle_data.asset_finder)


###########################################



# Create a screen for our Pipeline
universe = AverageDollarVolume(window_length = 60).top(1500)
# or single assets
#assets = bundle_data.asset_finder.lookup_symbols(['AAPL', 'MSFT'], as_of_date=None)
#sids = pd.Int64Index([asset.sid for asset in assets])


# Set the start and end dates
start_date = pd.Timestamp('2011-01-05', tz = 'utc')
end_date = pd.Timestamp('2017-02-01', tz = 'utc')



def make_pipeline(universe):

    # Momentum Factor Example
    momentum = Returns(inputs=[USEquityPricing.close], 
                        window_length=125,
                        mask=universe)

    last_close = USEquityPricing.close.latest
    
    SMA_200 = SimpleMovingAverage(inputs=[USEquityPricing.close],window_length=250,mask=universe)
    
    SMA_200_long = (last_close > SMA_200)
    
    SMA_200_short = (last_close < SMA_200)
    

    RSI_4 = RSI(inputs=[USEquityPricing.close],window_length=4,mask=universe)
    
    RSI_long = (RSI_4 < 20)
    
    RSI_short = (RSI_4 > 80)
    
    long = SMA_200_long & RSI_long & momentum.top(20)
    
    short = SMA_200_short & RSI_short & momentum.bottom(20)
    
    top_bottom_scores = ( long | short )
         
    return Pipeline(columns={
                              'MyFactor': momentum,
                              'longs': long,
                              'shorts': short,
                          }, screen=(universe & top_bottom_scores) )



# Execute pipeline over evaluation period

factors = engine.run_pipeline(make_pipeline(universe),start_date=start_date,end_date=end_date)
#factors = PipelineEngine.run_pipeline(initialize(universe),start_date=start_date,end_date=end_date)

factors = factors.dropna()

asset_list = factors.index.levels[1].unique()

prices = get_pricing(trading_calendar, asset_list, start_date=start_date, end_date=end_date )
print('done')

In [ ]:
factor_data = al.utils.get_clean_factor_and_forward_returns(
    factor=factors["MyFactor"],
    prices=prices,
    quantiles=5,
#    quantiles=None,
#    bins=2,
    periods=(1,5,10,20,40,80))

al.tears.create_full_tear_sheet(factor_data, long_short=True, group_neutral=False, by_group=False)